# Job Search Agent Tools Testing

This notebook tests all 4 tools available for the Job Search Agent:
1. **search_linkedin_jobs** - LinkedIn job search with filters
2. **parse_cv_content** - CV/Resume parsing from text or PDF

## Setup

In [1]:
import sys
import os

# Add project root and backend directories to Python path
project_root = os.path.dirname(os.getcwd())
backend_path = os.path.join(project_root, 'backend')
tester_path = os.getcwd()

for path in [project_root, backend_path, tester_path]:
    if path not in sys.path:
        sys.path.append(path)

print(f"Project root: {project_root}")
print(f"Backend path: {backend_path}")
print(f"Current working directory: {os.getcwd()}")

try:
    from config import MODEL_NAME
    print(f"Using model: {MODEL_NAME}")
except ImportError as e:
    print(f"Could not import config. Error: {e}")
    print("Please ensure config.py exists in one of these locations:")
    for path in sys.path:
        print(f"- {path}")

Project root: /home/duybaohuynhtan/Resume_Analyzer
Backend path: /home/duybaohuynhtan/Resume_Analyzer/backend
Current working directory: /home/duybaohuynhtan/Resume_Analyzer/tester
Using model: llama-3.1-8b-instant


In [2]:
# Import Job Search Agent
try:
    from job_search_agent import create_linkedin_job_agent
    
    print("✅ Job Search Agent imported successfully!")
    print(f"create_linkedin_job_agent: {create_linkedin_job_agent}")
    
    # Create the agent
    agent = create_linkedin_job_agent()
    print("✅ Agent created successfully!")
    print(f"Agent: {agent}")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running this from the tester directory")
except Exception as e:
    print(f"❌ Error creating agent: {e}")


✅ Job Search Agent imported successfully!
create_linkedin_job_agent: <function create_linkedin_job_agent at 0x7f71544afd80>
Creating agent with model: llama-3.1-8b-instant
✅ Agent created successfully!
Agent: memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history') verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_functions(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[lan

/home/duybaohuynhtan/Resume_Analyzer/backend/job_search_agent.py:90: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [3]:
# Load environment variables
from dotenv import load_dotenv
import json

load_dotenv()

# Check if API keys are available
api_key = os.getenv('CUSTOM_SEARCH_API_KEY')
search_engine_id = os.getenv('GOOGLE_SEARCH_ENGINE_ID')

if api_key and search_engine_id:
    print("✅ API credentials found")
else:
    print("⚠️ API credentials not found. Some tests may fail.")


✅ API credentials found


## Test Data Setup

Let's create some sample CV content for testing

In [ ]:
# Sample CV content for testing
sample_cv_text = """
John Doe
Senior Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: +1-555-123-4567
LinkedIn: linkedin.com/in/johndoe
Location: San Francisco, CA

Professional Summary:
Experienced software engineer with 7+ years of experience in full-stack development, 
specializing in Python, JavaScript, and cloud technologies. Proven track record of 
leading development teams and delivering scalable web applications.

Work Experience:

Senior Software Engineer | TechCorp Inc. | 2021 - Present
San Francisco, CA
- Led a team of 5 developers in building microservices architecture
- Implemented CI/CD pipelines using Docker and Kubernetes
- Developed RESTful APIs using Python Django and FastAPI
- Collaborated with product team to deliver features for 1M+ users

Software Engineer | StartupXYZ | 2019 - 2021
Remote
- Built responsive web applications using React and Node.js
- Designed and implemented PostgreSQL database schemas
- Integrated third-party APIs and payment systems
- Mentored junior developers and conducted code reviews

Junior Developer | WebSolutions Ltd. | 2017 - 2019
New York, NY
- Developed front-end components using HTML, CSS, and JavaScript
- Maintained legacy PHP applications
- Participated in agile development processes

Education:
Bachelor of Science in Computer Science
University of California, Berkeley | 2013 - 2017
GPA: 3.7/4.0

Technical Skills:
- Programming Languages: Python, JavaScript, TypeScript, Java, PHP
- Frameworks: Django, FastAPI, React, Node.js, Express
- Databases: PostgreSQL, MySQL, MongoDB, Redis
- Cloud: AWS, Docker, Kubernetes, Terraform
- Tools: Git, Jenkins, JIRA, VS Code

Certifications:
- AWS Certified Solutions Architect (2022)
- Certified Kubernetes Administrator (2021)

Preferred Work Arrangement: Remote or Hybrid
Expected Salary: $120,000 - $150,000
"""

print("Sample CV text prepared:")
print(f"Length: {len(sample_cv_text)} characters")
print(f"Lines: {len(sample_cv_text.split('\\n'))} lines")


## 1. Test Agent CV Parsing Capability

Testing the agent's ability to parse CVs when user provides CV content

In [ ]:
print("🧪 Testing Agent CV Parsing Capability")
print("=" * 50)

try:
    # Test 1: User asks agent to analyze their CV
    user_query_1 = f"""Please analyze my CV and tell me about my background:

{sample_cv_text}

Can you extract my key information including experience, skills, and career level?"""
    
    print("Test 1: CV Analysis Request")
    print("Query: 'Please analyze my CV and tell me about my background...'")
    print("\n🤖 Agent Response:")
    
    response_1 = agent.invoke({"input": user_query_1})
    print(response_1.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during agent CV analysis: {e}")

print("\n" + "="*60)


## 2. Test Agent Job Search Capability

Testing the agent's ability to search for jobs based on user criteria

In [ ]:
print("🧪 Testing Agent Job Search Capability")
print("=" * 50)

try:
    # Test 1: Basic job search request
    user_query_2 = """I'm looking for Python Developer jobs in San Francisco, CA. 
    Can you help me find some opportunities? Please show me a few relevant positions with details about the companies, requirements, and application links."""
    
    print("Test 1: Basic Job Search Request")
    print("Query: 'I'm looking for Python Developer jobs in San Francisco, CA...'")
    print("\n🤖 Agent Response:")
    
    response_2 = agent.invoke({"input": user_query_2})
    print(response_2.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during agent job search: {e}")

In [ ]:
print("🧪 Testing Agent Job Search Capability")
print("=" * 50)

try:
    # Test 1: Basic job search request
    user_query_2 = """I'm looking for Python Developer jobs in San Francisco, CA. 
    Can you help me find some opportunities? Please show me a few relevant positions with details about the companies, requirements, and application links."""
    
    print("Test 1: Basic Job Search Request")
    print("Query: 'I'm looking for Python Developer jobs in San Francisco, CA...'")
    print("\n🤖 Agent Response:")
    
    response_2 = agent.invoke({"input": user_query_2})
    print(response_2.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during agent job search: {e}")

print("\n" + "="*60)

try:
    # Test 2: Advanced job search with filters
    user_query_3 = """I want to find remote Senior Software Engineer positions. 
    I'm specifically interested in full-time roles at technology companies. 
    Can you search for recent job postings from the past week? Please find 3-4 relevant opportunities."""
    
    print("Test 2: Advanced Job Search with Filters")
    print("Query: 'I want to find remote Senior Software Engineer positions...'")
    print("\n🤖 Agent Response:")
    
    response_3 = agent.invoke({"input": user_query_3})
    print(response_3.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during advanced job search: {e}")

print("\n" + "="*60)


## 4. Test Agent PDF Handling and Career Guidance

Testing the agent's ability to handle PDF files and provide career advice

In [ ]:
print("🧪 Testing Agent PDF Handling and Career Guidance")
print("=" * 60)

try:
    # Test 1: Career advice request
    user_query_5 = """I'm a software engineer with 7+ years of experience in Python and JavaScript. 
    I want to transition to a more senior role, possibly as a tech lead or architect. 
    What should I focus on to make this transition? And can you help me find relevant positions?"""
    
    print("Test 1: Career Guidance Request")
    print("Query: 'I want to transition to a more senior role, possibly as a tech lead...'")
    print("\n🤖 Agent Response:")
    
    response_5 = agent.invoke({"input": user_query_5})
    print(response_5.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during career guidance: {e}")

print("\n" + "="*60)

try:
    # Test 2: PDF file handling (conceptual test)
    user_query_6 = """If I had a PDF resume file, how would you help me analyze it and find jobs? 
    What's the process you would follow? Can you explain your capabilities for handling PDF resumes?"""
    
    print("Test 2: PDF Handling Capabilities")
    print("Query: 'If I had a PDF resume file, how would you help me analyze it...'")
    print("\n🤖 Agent Response:")
    
    response_6 = agent.invoke({"input": user_query_6})
    print(response_6.get("output", "No response"))
    
except Exception as e:
    print(f"❌ Exception during PDF capabilities inquiry: {e}")

print("\n" + "="*60)
